In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Data visualization

import matplotlib
import matplotlib.pyplot as plt
import pydicom

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df =  pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_train.csv')
test_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/stage_1_sample_submission.csv')
dir_train= '../input/rsna-intracranial-hemorrhage-detection/stage_1_train_images/'
dir_input='/kaggle/input/'

# Exploratory analysis

In [ ]:
print("Train :",train_df.shape)
print("Test :",test_df.shape)

In [ ]:
display(train_df.head())

In [ ]:
display(train_df.tail())

In [ ]:
train_df.head()

In [ ]:
train_df.count()

In [ ]:
train_df['Image_ID'] = train_df['ID'].str.rsplit(pat='_',n=1,expand=True)[0]
train_df['Hemorrhage'] = train_df['ID'].str.rsplit(pat='_',n=1,expand=True)[1]
train_df=train_df[['Image_ID','Hemorrhage','Label']]

In [ ]:
train_df.head()

In [ ]:
#Nombre d'images uniques
print("Number of images :",train_df['Image_ID'].nunique())
print("Number of Hemorraghes :",train_df['Hemorrhage'].nunique())

In [ ]:
pd.DataFrame(train_df['Image_ID'].value_counts()).reset_index().head(10)

# Data visualization

##### An additional label for any, which should always be true if any of the sub-type labels is true. We could know the number of images that have any kind of hemorrhage with this variable

In [ ]:
pourcentage = train_df[(train_df['Hemorrhage']=='any')&(train_df['Label']==1)]['Image_ID'].count()/train_df['Image_ID'].nunique()*100
print("Pourtage d'images avec un type d'hemorragie : ",round(pourcentage,2),'%')

pd.DataFrame([pourcentage,100-pourcentage],columns=['Pourcentage']).plot(kind='pie',y='Pourcentage',
                                                                  labels=['Hemorrhage','Non_Hemorrhage'],title='Repartition Hemorrhage',
                                                                  autopct='%.1f%%',figsize=(6,6),shadow=True, startangle=90)
plt.show()

In [ ]:
Hemorrage = pd.DataFrame(train_df[(train_df['Label']==1)&(train_df['Hemorrhage']!='any')]['Hemorrhage'].value_counts()).reset_index()
Hemorrage.columns = ['Hemorrhage','Number_Pictures']

Hemorrage.plot(kind='pie',y='Number_Pictures',labels=Hemorrage['Hemorrhage'].unique(),title='Repartition Hemorrhage',
                                                                  autopct='%.1f%%',figsize=(6,6),shadow=True, startangle=90)
plt.show()

In [ ]:
train_df.head()

In [ ]:
train_df.head()

In [ ]:
ds = pydicom.dcmread(dir_train+train_df['Image_ID'][0]+".dcm")
display(ds)
plt.imshow(ds.pixel_array, cmap=plt.cm.bone)

In [ ]:
def view_images(data_frame,hemorraghe):
    width = 5
    height = 1
    fig, axs = plt.subplots(height, width, figsize=(20,5))

    list_hem = pd.DataFrame(train_df[(train_df['Label']==1)&(train_df['Hemorrhage']==hemorraghe)][['Image_ID']].head(width*height)+".dcm").reset_index()
    
    for i in range(0,width*height):
        ds = pydicom.dcmread(dir_train+list_hem['Image_ID'][i])
        fig.add_subplot(height,width, i+1)
        axs[i].set_title(list_hem['Image_ID'][i])
        plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
        
    plt.suptitle("Images with "+hemorraghe,fontsize = 20)
    plt.show()

In [ ]:
for i in train_df['Hemorrhage'].unique():
    view_images(train_df,i)

In [ ]:
pivot_df = train_df.drop_duplicates().pivot(index='Image_ID', columns='Hemorrhage', values='Label').reset_index()
pivot_df.shape
pivot_df.head()

# Model

In [5]:
from keras import layers
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
from tqdm import tqdm
import torch
import keras


Using TensorFlow backend.


In [6]:
densenet = DenseNet121(
    weights='../input/densenet-keras/DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

KeyboardInterrupt: 

In [ ]:
# if not os.path.exists(dir_input+'DenseNet/'):
#     os.mkdir(dir_input+'DenseNet/')
#     print("Directory Created ")
# else:    
#     print("Directory already exists")
    
# !git clone https://github.com/titu1994/DenseNet.git /kaggle/input/DenseNet/